In [1]:
import sys, os
sys.path.append("../../models/PFIN")
sys.path.append(os.path.abspath('../../fastjet-install/lib/python3.9/site-packages'))
from dataset_pfn import PFNDataset
import numpy as np
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from PFIN import PFIN as Model
import sklearn
from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve
import json
from scipy import interpolate
from matplotlib import pyplot as plt
%env HDF5_USE_FILE_LOCKING=FALSE

env: HDF5_USE_FILE_LOCKING=FALSE


In [2]:
all_models = [f for f in os.listdir("../../models/PFIN/trained_models") if "_best" in f]
print("\n".join(all_models))

PFIN_best_v3_sum
PFIN_best_v3_cat


In [3]:
test_path = "../../datasets/test.h5"

#Loading testing dataset
test_set = PFNDataset(test_path, n_constits = 60, preprocessed=True)
testloader = DataLoader(test_set, shuffle=True, batch_size=512, num_workers=8, pin_memory=True, persistent_workers=True)

In [4]:
def eval2(model):
    labels = []
    preds = []
    with torch.no_grad():
        for x,m,y,a,t in tqdm(testloader):
            x = x.cuda()
            m = m.cuda()
            a = a.cuda()
            t = t.cuda()
            pred = model(x, a, m, t)
            labels.append(y[:,1].cpu().numpy())
            preds.append(pred[:,1].cpu().numpy())
    labels = np.concatenate(labels, axis=None)
    preds = np.concatenate(preds, axis=None)
    return labels, preds

In [5]:
all_models = ['PFIN_best_v3_sum', 'PFIN_best_v3_cat']
for modelname in all_models:
    model_dict = json.load(open("../../models/PFIN/trained_model_dicts/" + modelname.replace("_best","") + ".json"))
    label = model_dict['label']
    f_nodes = list(map(int, model_dict['f_nodes'].split(',')))
    phi_nodes = list(map(int, model_dict['phi_nodes'].split(',')))
    augmented = model_dict['augmented']

    model = Model(particle_feats=3,
                  interaction_feats=4,
                  Phi_sizes=phi_nodes, 
                  F_sizes=f_nodes,
                  PhiI_nodes = model_dict['n_phiI'],
                  n_consts = 60,
                  interaction_mode = model_dict['x_mode'],
                  augmented = augmented).cuda()
    model.load_state_dict(torch.load("../../models/PFIN/trained_models/" + modelname ))
    nparams = sum(p.numel() for p in model.parameters())
    
    # FcLayers = []
    # PhiLayers = []
    # for layer in model._modules['fc']:
    #     FcLayers.append(layer)
    # for layer in model._modules['phi']:
    #     PhiLayers.append(layer)
    
    
    labels, preds = eval2(model)
    accuracy = accuracy_score(labels, preds.round())*100
    
    auc = roc_auc_score(labels, preds)*100
    
    fpr, tpr, thresholds = roc_curve(labels, preds, drop_intermediate=False)
    intrp = interpolate.interp1d(tpr, fpr)
    if 'baseline' in modelname:
        eS = np.array(list(range(1,20)))*0.05
        inveB = []
        for es in eS:
            inveB.append(1./intrp(es))
        plt.figure()
        plt.plot(eS, inveB)
        plt.yscale("log")
        plt.show()
    brr = 1./intrp(0.3)
    print("{} \t\t Params: {}\t ROC-AUC: {:.2f}%, Accuracy: {:.2f}%, BRR: {:.2f}".format(modelname,nparams,auc,accuracy,brr))

[100, 100, 64] [64, 100, 100]


100%|██████████| 790/790 [00:27<00:00, 28.47it/s] 


PFIN_best_v3_sum 		 Params: 96806	 ROC-AUC: 98.39%, Accuracy: 93.68%, BRR: 1040.79
[100, 100, 64] [64, 100, 100]


100%|██████████| 790/790 [00:06<00:00, 113.70it/s]


PFIN_best_v3_cat 		 Params: 100902	 ROC-AUC: 98.38%, Accuracy: 93.70%, BRR: 1030.17
